In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
np.seterr(over='raise', invalid='raise', divide='raise');

## Load the training data into feature matrix, class labels, and event ids:

In [66]:
from proj1_helpers import *
from helpers import *
DATA_TRAIN_PATH = '../train.csv' # TODO: download train data and supply path here 
y, X, ids = load_csv_data(DATA_TRAIN_PATH)

In [67]:
from preprocessing import *
X0, X1, X2, X3, indices0, indices1, indices2, indices3 = separate_data(X)
y0 = y[indices0]
y1 = y[indices1]
y2 = y[indices2]
y3 = y[indices3]

overall_acc = []

In [55]:
from toolbox import *
# Define the parameters of the algorithm.
max_iters_0 = 13000
gamma_0 = 1e-6
lambda_0 = 0
degree_0 = 2
batch_size0 = 100
k = 4
current_y = y0
current_X = X0
k_indices = build_k_indices(current_y, k, 1)

loss_tr = []
loss_te = []
classification_acc = []
for k_ in range(k):

    test_indices = k_indices[k_]
    test_y, test_x = (current_y[test_indices], current_X[test_indices])
    test_x = build_poly(test_x, degree_0)

    training_indices = np.ravel(np.delete(k_indices, k_, axis=0))
    training_y, training_x = (current_y[training_indices], current_X[training_indices])
    training_x = build_poly(training_x, degree_0)

    w_star = reg_logistic_regression(training_y, training_x, lambda_0, gamma0, max_iters0)
    
    # compute classification accuracy
    y_pred = predict_labels_log_regression(w_star, test_x)
    
    test_y[test_y == 0] = -1
    classification_acc.append(np.mean(np.abs(test_y + y_pred) / 2))
    test_y[test_y == -1] = 0

    loss_tr.append(calculate_loss_log_likelihood(training_y, training_x, w_star))
    loss_te.append(calculate_loss_log_likelihood(test_y, test_x, w_star))
        
print("Training error: {tr}\nTest error: {te}\nClassification accuracy: {cl}".format(tr=np.mean(loss_tr), te=np.mean(loss_te), cl=np.mean(classification_acc)))
overall_acc.append(np.mean(classification_acc))

Current iteration=0, the loss=46187.23525946155
Current iteration=100, the loss=29608.044414346696
Current iteration=200, the loss=29186.166471384226
Current iteration=300, the loss=29017.842782262862
Current iteration=400, the loss=28910.925973800644
Current iteration=500, the loss=28829.3666118864
Current iteration=600, the loss=28761.120612035236
Current iteration=700, the loss=28701.188581964307
Current iteration=800, the loss=28647.146053965014
Current iteration=900, the loss=28597.644863433816
Current iteration=1000, the loss=28551.84582331616
Current iteration=1100, the loss=28509.179949521305
Current iteration=1200, the loss=28469.236541813694
Current iteration=1300, the loss=28431.704977550646
Current iteration=1400, the loss=28396.341573961494
Current iteration=1500, the loss=28362.94932890321
Current iteration=1600, the loss=28331.364867562843
Current iteration=1700, the loss=28301.449708075776
Current iteration=1800, the loss=28273.08425201797
Current iteration=1900, the lo

In [58]:
from toolbox import *
# Define the parameters of the algorithm.
max_iters_1 = 12000
gamma_1 = 1e-7
lambda_1 = 0
degree_1 = 3
batch_size1 = 100
k = 4
current_y = y1
current_X = X1
k_indices = build_k_indices(current_y, k, 1)

loss_tr = []
loss_te = []
classification_acc = []
for k_ in range(k):

    test_indices = k_indices[k_]
    test_y, test_x = (current_y[test_indices], current_X[test_indices])
    test_x = build_poly(test_x, degree_1)

    training_indices = np.ravel(np.delete(k_indices, k_, axis=0))
    training_y, training_x = (current_y[training_indices], current_X[training_indices])
    training_x = build_poly(training_x, degree_1)

    w_star = reg_logistic_regression(training_y, training_x, lambda_1, gamma_1, max_iters_1)
    
    # compute classification accuracy
    y_pred = predict_labels_log_regression(w_star, test_x)
    
    test_y[test_y == 0] = -1
    classification_acc.append(np.mean(np.abs(test_y + y_pred) / 2))
    test_y[test_y == -1] = 0

    loss_tr.append(calculate_loss_log_likelihood(training_y, training_x, w_star))
    loss_te.append(calculate_loss_log_likelihood(test_y, test_x, w_star))
        
print("Training error: {tr}\nTest error: {te}\nClassification accuracy: {cl}".format(tr=np.mean(loss_tr), te=np.mean(loss_te), cl=np.mean(classification_acc)))
overall_acc.append(np.mean(classification_acc))

Current iteration=0, the loss=39301.053110612665
Current iteration=100, the loss=33887.81607392824
Current iteration=200, the loss=32770.19875464415
Current iteration=300, the loss=31300.689909847337
Current iteration=400, the loss=30994.446457904032
Current iteration=500, the loss=30499.38100425284
Current iteration=600, the loss=30051.542864848438
Current iteration=700, the loss=29648.387436684185
Current iteration=800, the loss=29359.77393393892
Current iteration=900, the loss=29136.008401918
Current iteration=1000, the loss=28947.158543872025
Current iteration=1100, the loss=28785.561886004572
Current iteration=1200, the loss=28645.686168554283
Current iteration=1300, the loss=28523.43557767925
Current iteration=1400, the loss=28415.717546463144
Current iteration=1500, the loss=28320.133926955812
Current iteration=1600, the loss=28234.77810288683
Current iteration=1700, the loss=28158.11189697717
Current iteration=1800, the loss=28088.884942615703
Current iteration=1900, the loss=2

In [59]:
from toolbox import *
# Define the parameters of the algorithm.
max_iters_2 = 12000
gamma_2 = 1e-6
lambda_2 = 0.0
degree_2 = 2
batch_size2 = 100
k = 2
current_y = y2
current_X = X2
k_indices = build_k_indices(current_y, k, 1)

loss_tr = []
loss_te = []
classification_acc = []
for k_ in range(k):

    test_indices = k_indices[k_]
    test_y, test_x = (current_y[test_indices], current_X[test_indices])
    test_x = build_poly(test_x, degree_2)

    training_indices = np.ravel(np.delete(k_indices, k_, axis=0))
    training_y, training_x = (current_y[training_indices], current_X[training_indices])
    training_x = build_poly(training_x, degree_2)
    
    w_star = reg_logistic_regression(training_y, training_x, lambda_2, gamma_2, max_iters_2)
    
    # compute classification accuracy
    y_pred = predict_labels_log_regression(w_star, test_x)
    
    test_y[test_y == 0] = -1
    classification_acc.append(np.mean(np.abs(test_y + y_pred) / 2))
    test_y[test_y == -1] = 0

    loss_tr.append(calculate_loss_log_likelihood(training_y, training_x, w_star))
    loss_te.append(calculate_loss_log_likelihood(test_y, test_x, w_star))
        
print("Training error: {tr}\nTest error: {te}\nClassification accuracy: {cl}".format(tr=np.mean(loss_tr), te=np.mean(loss_te), cl=np.mean(classification_acc)))
overall_acc.append(np.mean(classification_acc))

Current iteration=0, the loss=17029.074232293548
Current iteration=100, the loss=12857.640260643777
Current iteration=200, the loss=12290.531448393613
Current iteration=300, the loss=11977.920522161216
Current iteration=400, the loss=11768.22536106244
Current iteration=500, the loss=11617.056287885822
Current iteration=600, the loss=11503.492068653803
Current iteration=700, the loss=11415.749822105838
Current iteration=800, the loss=11346.547002092879
Current iteration=900, the loss=11291.09383232849
Current iteration=1000, the loss=11246.092335861133
Current iteration=1100, the loss=11209.190944059956
Current iteration=1200, the loss=11178.666323994246
Current iteration=1300, the loss=11153.227300958104
Current iteration=1400, the loss=11131.88863435428
Current iteration=1500, the loss=11113.886863402193
Current iteration=1600, the loss=11098.62255934523
Current iteration=1700, the loss=11085.619721414074
Current iteration=1800, the loss=11074.496621884617
Current iteration=1900, the 

In [60]:
from toolbox import *
# Define the parameters of the algorithm.
max_iters_3 = 12000
gamma_3 = 1e-6
lambda_3 = 0
degree_3 = 3
batch_size3 = 100
k = 2
current_y = y3
current_X = X3
k_indices = build_k_indices(current_y, k, 1)

loss_tr = []
loss_te = []
classification_acc = []
for k_ in range(k):

    test_indices = k_indices[k_]
    test_y, test_x = (current_y[test_indices], current_X[test_indices])
    test_x = build_poly(test_x, degree_3)

    training_indices = np.ravel(np.delete(k_indices, k_, axis=0))
    training_y, training_x = (current_y[training_indices], current_X[training_indices])
    training_x = build_poly(training_x, degree_3)

    w_star = reg_logistic_regression(training_y, training_x, lambda_3, gamma_3, max_iters_3)
    
    # compute classification accuracy
    y_pred = predict_labels_log_regression(w_star, test_x)
    
    test_y[test_y == 0] = -1
    classification_acc.append(np.mean(np.abs(test_y + y_pred) / 2))
    test_y[test_y == -1] = 0

    loss_tr.append(calculate_loss_log_likelihood(training_y, training_x, w_star))
    loss_te.append(calculate_loss_log_likelihood(test_y, test_x, w_star))
        
print("Training error: {tr}\nTest error: {te}\nClassification accuracy: {cl}".format(tr=np.mean(loss_tr), te=np.mean(loss_te), cl=np.mean(classification_acc)))
overall_acc.append(np.mean(classification_acc))

Current iteration=0, the loss=7263.42224758955
Current iteration=100, the loss=6048.835832145423
Current iteration=200, the loss=5896.717748681341
Current iteration=300, the loss=5760.360737329438
Current iteration=400, the loss=5675.60681034956
Current iteration=500, the loss=5608.947593023511
Current iteration=600, the loss=5538.4055733654
Current iteration=700, the loss=5479.672266947327
Current iteration=800, the loss=5445.250065851807
Current iteration=900, the loss=5400.945775770389
Current iteration=1000, the loss=5358.6704474901635
Current iteration=1100, the loss=5329.941929169332
Current iteration=1200, the loss=5306.425869978817
Current iteration=1300, the loss=5276.600837577902
Current iteration=1400, the loss=5269.784719560236
Current iteration=1500, the loss=5239.125540447688
Current iteration=1600, the loss=5237.472083371727
Current iteration=1700, the loss=5198.917012250523
Current iteration=1800, the loss=5199.71178160437
Current iteration=1900, the loss=5174.988139311

In [61]:
print("The overall accuracy: {acc}\n".format(acc=np.mean(overall_acc)))

The overall accuracy: 0.8007403114973887



In [69]:
from preprocessing import *

gamma_0 = 1e-6
max_iters_0 = 15000
degree_0 = 2

max_iters_1 = 15000
gamma_1 = 1e-7
degree_1 = 3

max_iters_2 = 15000
gamma_2 = 1e-6
degree_2 = 2

max_iters_3 = 15000
gamma_3 = 1e-6
degree_3 = 3

X0, X1, X2, X3, indices0, indices1, indices2, indices3 = separate_data(X)
y0 = y[indices0]
y1 = y[indices1]
y2 = y[indices2]
y3 = y[indices3]


phi_X0 = build_poly(X0, degree_0)
phi_X1 = build_poly(X1, degree_1)
phi_X2 = build_poly(X2, degree_2)
phi_X3 = build_poly(X3, degree_3)

w0 = reg_logistic_regression(y0, phi_X0, 0, gamma_0, max_iters_0)
w1 = reg_logistic_regression(y1, phi_X1, 0, gamma_1, max_iters_1)
w2 = reg_logistic_regression(y2, phi_X2, 0, gamma_2, max_iters_2)
w3 = reg_logistic_regression(y3, phi_X3, 0, gamma_3, max_iters_3)

Current iteration=0, the loss=59827.901962343414
Current iteration=100, the loss=39310.18955146214
Current iteration=200, the loss=38865.471345088095
Current iteration=300, the loss=38660.74811632965
Current iteration=400, the loss=38519.46500371248
Current iteration=500, the loss=38405.90285230931
Current iteration=600, the loss=38308.50444148451
Current iteration=700, the loss=38222.30110685889
Current iteration=800, the loss=38144.637780473335
Current iteration=900, the loss=38073.87952268947
Current iteration=1000, the loss=38008.924356192074
Current iteration=1100, the loss=37948.9808399343
Current iteration=1200, the loss=37893.45154015081
Current iteration=1300, the loss=37841.866452641756
Current iteration=1400, the loss=37793.84284052694
Current iteration=1500, the loss=37749.06007715472
Current iteration=1600, the loss=37707.243401326166
Current iteration=1700, the loss=37668.1531276278
Current iteration=1800, the loss=37631.57727374803
Current iteration=1900, the loss=37597.

## Generate predictions and save ouput in csv format for submission:

In [70]:
DATA_TEST_PATH = '../test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)
X0_te, X1_te, X2_te, X3_te, indices0_te, indices1_te, indices2_te, indices3_te = separate_data(tX_test)

In [72]:
OUTPUT_PATH = '../submission.csv' # TODO: fill in desired name of output file for submission
ids_test0 = ids_test[indices0_te]
ids_test1 = ids_test[indices1_te]
ids_test2 = ids_test[indices2_te]
ids_test3 = ids_test[indices3_te]

phi_X0_te = build_poly(X0_te, degree_0)
phi_X1_te = build_poly(X1_te, degree_1)
phi_X2_te = build_poly(X2_te, degree_2)
phi_X3_te = build_poly(X3_te, degree_3)

pred0 = predict_labels_log_regression(w0, phi_X0_te)
pred1 = predict_labels_log_regression(w1, phi_X1_te)
pred2 = predict_labels_log_regression(w2, phi_X2_te)
pred3 = predict_labels_log_regression(w3, phi_X3_te)

predictions = np.append(pred0, np.append(pred1, np.append(pred2, pred3)))
ids = np.append(ids_test0, np.append(ids_test1, np.append(ids_test2, ids_test3)))

create_csv_submission(ids, predictions, OUTPUT_PATH)
